In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install datasets -U

In [ ]:
import pandas as pd
df = pd.read_csv('/kaggle/input/politifact/synthetic-gpt-3.5-turbo_politifact_paraphrase_generation_processed.csv')

In [ ]:
from sklearn.model_selection import train_test_split
train_df, test_df, _,_ = train_test_split(df, df["label"], test_size=0.2, stratify=df['label'],random_state=42)

In [ ]:
def process_data(train_df):
    train_data = []
    for i in range(len(train_df)):
        train_data.append({'label':train_df.iloc[i]['label'],'text':train_df.iloc[i]['synthetic_misinformation'][:1000]})
    return train_data

In [ ]:
train_data = process_data(train_df)
test_data = process_data(test_df)

In [ ]:
# from trl import SFTTrainer
from datasets import Dataset
dataset = Dataset.from_list(train_data[:])
dataset_eval = Dataset.from_list(test_data[:])

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets_train = dataset.map(tokenize_function, batched=True)
tokenized_datasets_eval = dataset_eval.map(tokenize_function, batched=True)

In [ ]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("caballeroch/FakeNewsClassifierDistilBert-uncased", num_labels=2)
# model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

In [ ]:
!pip install evaluate

In [ ]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch",num_train_epochs=3)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets_train,
    eval_dataset=tokenized_datasets_eval,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.evaluate(eval_dataset=tokenized_datasets_eval)

In [ ]:
trainer.train()

In [ ]:
test_df_h = pd.read_csv('/kaggle/input/politifact/synthetic-gpt-3.5-turbo_politifact_partially_arbitrary_generation_politics_rumors_processed.csv')

In [ ]:
def process_data_h(train_df):
    train_data = []
    for i in range(len(train_df)):
        train_data.append({'label':train_df.iloc[i]['label'],'text':train_df.iloc[i]['synthetic misinformation'][:1000]})
    return train_data

In [ ]:
test_data_h = process_data_h(test_df_h)
test_data_h = Dataset.from_list(test_data_h[:])
tokenized_datasets_th = test_data_h.map(tokenize_function, batched=True)
trainer.evaluate(eval_dataset=tokenized_datasets_th)

In [ ]:
test_df_h = pd.read_csv('/kaggle/input/politifact/synthetic-gpt-3.5-turbo_politifact_partially_arbitrary_generation_politics_rumors_processed.csv')

In [ ]:
test_df_h.head()

In [ ]:
supervised = []
prompt = "Given a 'passage' determine whether or not it is a piece of misinformation. ONLY output 'YES' or 'NO'. The 'passage' is: "
count=0
for n in list(test_df["synthetic_misinformation"]):
#     print(f"{prompt}{n}")

    pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=2000)
    result = pipe(f"<s> [INST] {prompt}{n} [/INST]")
    supervised.append(result[0]['generated_text'])
    count+=1
    print(count)


In [ ]:
sft_res = []
for x in supervised:
#     sft_res.
    if(x[-3:].strip()=='NO'):
        sft_res.append(0)
    else:
        sft_res.append(1)
    

In [ ]:
test_df_h['label'].value_counts()

In [ ]:
# sft_res

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(test_df['label'].values, sft_res)

In [ ]:
count